In [14]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation

with open("C:\Users\User\Desktop\BIOINFORMATICA\id_seq.txt", "r"):
    gen11 = gen1.read()
    gen12 = ",".join(gen11.split("\n")).split(",")


<class 'SyntaxError'>: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-14-e79bdcfc8dc5>, line 5)